In [8]:
import cv2
import numpy as np

video_path = 'test.mjpeg'
cap = cv2.VideoCapture(video_path)

frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frames.append(gray_frame)

cap.release()

# Convert the list of frames into a 3D numpy array (X x Y x Time)

video_array = np.stack(frames, axis=-1)

# video_array is the np array for each frame:

X1, X2 = 0,640
Y1, Y2 = 0,480  #640x480


# Extract this region for all time frames


# To find diff between frames: use stream.diff 

#stream_1_field = video_array[X1:X2, Y1:Y2, :]
#stream_diff = np.diff(stream_1_field, axis=-1)



In [9]:
video_array

array([[[ 21,  21,  20, ...,  25,  18,  23],
        [ 28,  20,  19, ...,  29,  17,  20],
        [ 24,  20,  18, ...,  25,  15,  16],
        ...,
        [138, 139, 137, ..., 134, 138, 137],
        [134, 138, 137, ..., 132, 136, 137],
        [132, 137, 136, ..., 131, 135, 136]],

       [[ 21,  20,  19, ...,  23,  18,  22],
        [ 27,  20,  18, ...,  27,  17,  20],
        [ 23,  19,  17, ...,  23,  15,  16],
        ...,
        [137, 139, 137, ..., 134, 138, 137],
        [135, 137, 136, ..., 132, 135, 136],
        [134, 136, 135, ..., 131, 134, 135]],

       [[ 21,  20,  17, ...,  19,  18,  21],
        [ 24,  19,  17, ...,  24,  17,  19],
        [ 22,  19,  15, ...,  21,  15,  17],
        ...,
        [135, 138, 136, ..., 134, 137, 136],
        [136, 136, 135, ..., 132, 134, 135],
        [136, 134, 134, ..., 131, 132, 134]],

       ...,

       [[ 27,  31,  29, ..., 127, 127, 127],
        [ 27,  29,  28, ..., 127, 127, 127],
        [ 27,  27,  26, ..., 127, 127, 127